In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch

This data is CGM values in mg/dl and is in 15 minute increments. Each row is a new patient and each column is a reading. Each patient has 480 readings or 5 days of data 

In [52]:
df = pd.read_csv("cgm_data.csv").sample(frac=1)
for column in df.columns:
    df[column] = df[column].astype(float)
df.rename({0:"ID"}, axis=1)
df

,ptid,0 days 00:00:00,0 days 00:15:00,0 days 00:30:00,0 days 00:45:00,0 days 01:00:00,0 days 01:15:00,0 days 01:30:00,0 days 01:45:00,0 days 02:00:00,...,4 days 21:30:00,4 days 21:45:00,4 days 22:00:00,4 days 22:15:00,4 days 22:30:00,4 days 22:45:00,4 days 23:00:00,4 days 23:15:00,4 days 23:30:00,4 days 23:45:00
45,2052.0,205.2,194.4,174.6,158.4,151.2,131.4,127.8,149.4,165.6,...,259.2,280.8,293.4,311.4,327.6,331.2,325.8,306.0,297.0,286.2
36,2039.0,225.0,232.2,239.4,246.6,246.6,246.6,246.6,239.4,235.8,...,194.4,192.6,192.6,192.6,192.6,189.0,189.0,194.4,194.4,192.6
69,2073.0,237.6,253.8,241.2,214.2,205.2,198.0,183.6,174.6,183.6,...,217.8,225.0,228.6,226.8,217.8,212.4,217.8,210.6,192.6,187.2
34,2037.0,145.8,135.0,127.8,122.4,115.2,106.2,100.8,97.2,102.6,...,99.0,102.6,109.8,115.2,115.2,113.4,115.2,124.2,129.6,126.0
61,2067.0,82.8,82.8,82.8,82.8,81.0,81.0,86.4,88.2,84.6,...,154.8,142.2,135.0,138.6,144.0,142.2,144.0,138.6,129.6,122.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,2088.0,190.8,223.2,246.6,257.4,273.6,291.6,307.8,320.4,331.2,...,158.4,163.8,172.8,172.8,174.6,183.6,187.2,192.6,190.8,190.8
41,2048.0,55.8,61.2,66.6,72.0,73.8,61.2,54.0,57.6,100.8,...,145.8,145.8,145.8,145.8,142.2,138.6,145.8,163.8,183.6,196.2
42,2049.0,266.4,280.8,311.4,329.4,331.2,334.8,345.6,354.6,361.8,...,250.2,262.8,271.8,289.8,300.6,297.0,286.2,273.6,264.6,255.6
1,2001.0,102.6,97.2,95.4,91.8,86.4,82.8,86.4,90.0,88.2,...,145.8,151.2,149.4,145.8,145.8,142.2,136.8,127.8,126.0,127.8


In [53]:
class LSTMPredictor(torch.nn.Module):
    def __init__(self,n_hidden=51, device='cpu'):
        super(LSTMPredictor, self).__init__()
        self.n_hidden = n_hidden
        # lstm1, lstm2, linear
        self.lstm1 = nn.LSTMCell(1,self.n_hidden)
        self.lstm2 = nn.LSTMCell(self.hidden,self.n_hidden)
        self.linear = nn.Linear(self.n_hidden,1)
    
    def forward(self,x,future=0):
        # N, 100
        outputs = []
        n_samples = x.size(0)
        h_t = torch.zeros(n_samples, self.n_hidden, dtype = torch.float32)
        c_t = torch.zeros(n_samples, self.n_hidden, dtype = torch.float32)
        
        
        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype = torch.float32)
        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype = torch.float32)
        
        for input_t in x.split(1, dim=1):
            #N, 1
            h_t, c_t = self.lstm1(input_t, (h_t,c_t))
            h_t2, c_t2 = self.lstm2(input_t, (h_t2,c_t2))
            output = self.linear(h_t2)
            outputs.append(output)
        
        for i in range(future):
            h_t, c_t = self.lstm1(output, (h_t,c_t))
            h_t2, c_t2 = self.lstm1(input_t, (h_t2,c_t2))
            output = self.linear(h_t2)
            outputs.append(output)
            
        outputs = torch.cat(outputs,dim=1)
        return outputs

In [14]:
df.iloc[:,1:-1].dtypes

1      object
2      object
3      object
4      object
5      object
        ...  
475    object
476    object
477    object
478    object
479    object
Length: 479, dtype: object

In [12]:
train_input = torch.from_numpy(df.iloc[:,1:-1].values)
train_input = torch.from_numpy(df.iloc[:,1:].values)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.